### The Relationship Between S&P 500 ESG Scores and Their Stock Prices (Annual Return)

In this project, the hypothesis being tested is whether ESG scores influence a company's stock price. The stock price data for the companies starts on January 3, 2023, and ends on August 30, 2024. The ESG scores for the companies were released in September 2023. 

>To understand the effect of ESG scores on stock prices, two approaches were considered: 
    - Either using the price difference between the start and end of the period, 
    - Or calculating the annual return (percentage growth) and using that data. 
After considering both options, the decision was made to use **annual return** for the analysis.

#### Price Difference vs. Annual Return

1. **Price Difference** shows only the absolute change in price.
   
   Example:

   | Company | Start Price | End Price | Price Difference |

   |---------|-------------|-----------|------------------|

   | AAPL    | 150         | 160       | +10 USD          |

   | TSLA    | 800         | 810       | +10 USD          |

   Here, both Apple and Tesla have the same price difference (+10 USD), but the significance of this difference is very different for investors:

   - For Apple: **+6.67% gain**
   - For Tesla: **+1.25% gain**

   Therefore, **Price Difference** does not account for the size of the company or its stock price!

2. **Annual Return** gives a normalized percentage change in value, allowing for fairer comparisons between companies of different sizes.

   Using the same data in terms of annual return:

   | Company | Start Price | End Price | Annual Return |

   |---------|-------------|-----------|---------------|

   | AAPL    | 150         | 160       | +6.67%        |

   | TSLA    | 800         | 810       | +1.25%        |

   Now, the size of the company or stock price is no longer important. The percentage return shows clearly which company has delivered a better return to its investors.

#### So, Why Use Annual Return?

- It **normalizes** differences between companies of various sizes and stock prices.
- It measures **actual investment performance**.
- It **standardizes interpretation**: regardless of company size, the percentage return is what's being discussed.
- **Statistical comparisons** (like t-tests and correlations) are more **fair and accurate**.

### Hypotheses:

- **H₀ (Null Hypothesis)**: There is no relationship between ESG scores and annual return.
- **Hₐ (Alternative Hypothesis)**: There is a relationship between ESG scores and annual return.


### Methods Used in This Analysis:

1. **Descriptive Statistics** (mean, median, variance, standard deviation, etc.) were first calculated to get a basic understanding of the data.
2. A **histogram** was used to observe the distribution of the data and check for any skewness or outliers in the ESG scores and annual returns.
3. A **scatter plot** was then created to visually observe any potential relationship between ESG scores and annual returns.
4. Finally, for hypothesis testing, the **Pearson correlation method** was used. This was chosen because Pearson correlation is ideal for measuring the linear relationship between two continuous variables, such as ESG scores and annual returns.


## Data Loading and Initial Setup

- **Essential Python libraries were imported:**
  - `NumPy` was used for numerical operations.
  - `Pandas` was used for handling tabular data and creating DataFrames.
  - `Matplotlib` and `Seaborn` were used for data visualization and plotting.
  - `SciPy` was used for performing statistical tests and calculations.

- **Two datasets were identified and used in the project:**
  - The first dataset includes **ESG scores** for S&P 500 companies.
  - The second dataset contains **historical stock prices** for the same companies.
  - NOTE: **Both datasets were obtained from Kaggle**, a popular online platform for open datasets, frequently used in data science and machine learning projects.

- **Both datasets were loaded into separate Pandas DataFrames:**
  - The `read_csv()` function was used to read the `.csv` files into memory.

- **Datetime parsing was applied to the stock price data:**
  - While loading the stock price dataset, the `Date` column was parsed as `datetime` objects.
  - This enables efficient time-based filtering and calculations such as selecting specific dates and computing annual returns.

- **After this step, the data was ready for further processing:**
  - This included merging ESG and stock price data, calculating returns, conducting exploratory data analysis (EDA), and performing hypothesis testing.


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy import special

esg_data = "sp500_esg_data.csv"
price_data = "sp500_price_data.csv"

df_esg = pd.read_csv(esg_data)
df_prices = pd.read_csv(price_data, parse_dates=["Date"])

###  Data Preparation and Preprocessing

- **Stock prices for two specific dates were selected:**
  - January 3, 2023 and August 30, 2024 were chosen as the start and end points of the analysis.
  - The corresponding rows from the stock price dataset were extracted.

- **Annual return for each company was calculated:**
  - The formula used was:  
    Annual Return = ((End Price / Start Price) - 1) * 100 
  - This provides a normalized percentage change in stock price, which allows for comparisons across companies with different price levels.

- **A new DataFrame was created to store return-related data:**
  - The DataFrame contains the following columns:  
    `Symbol`, `StartPrice`, `EndPrice`, and `AnnualReturn`.

- **A custom function was defined to compute descriptive statistics:**
  - The function `describe_return()` calculates key statistics such as:
    - Count, Min, Max, Range, Mean, Median, Variance, Standard Deviation, Skewness, and Kurtosis.
  - These are useful for comparing the distribution of returns between groups (e.g., High ESG vs. Low ESG).

- **The ESG dataset was merged with the return dataset:**
  - The `merge()` operation used the `"Symbol"` column as the key.
  - This created a combined DataFrame with ESG scores and return data for each company.

- **An additional column was created to calculate the absolute price difference:**
  - `PriceDifference = EndPrice - StartPrice`
  - This represents the nominal change in stock price but is not used for final analysis (AnnualReturn is preferred for normalization).

- **Only relevant columns were selected for further analysis:**
  - `Symbol`, `StartPrice`, `EndPrice`, `AnnualReturn`, `PriceDifference`, and `totalEsg`.

- **Numeric type enforcement was applied:**
  - The `pd.to_numeric()` function was used to convert string or mixed-type data into numeric values.
  - Errors were coerced into NaN to ensure data cleanliness.



In [ ]:
###  Data Preparation and Preprocessing

start_prices = df_prices[df_prices["Date"] == "2023-01-03"].iloc[0]
end_prices = df_prices[df_prices["Date"] == "2024-08-30"].iloc[0]
annual_return = (end_prices[1:] / start_prices[1:]) - 1

df_returns = pd.DataFrame({
    "Symbol": start_prices.index[1:],   # şirket sembolleri
    "StartPrice": start_prices[1:].values,
    "EndPrice": end_prices[1:].values,
    "AnnualReturn": annual_return.values
})

def describe_return(group, label):
    data = group['AnnualReturn'].dropna()
    stats_dict = {
        "Label": label,
        "Count": data.count(),
        "Min": data.min(),
        "Max": data.max(),
        "Range": data.max() - data.min(),
        "Mean": data.mean(),
        "Median": data.median(),
        "Variance": data.var(),
        "Std Dev": data.std(),
        "Skewness": stats.skew(data),
        "Kurtosis": stats.kurtosis(data)
    }
    return pd.DataFrame([stats_dict])



df_final = df_esg.merge(df_returns, on="Symbol")
df_final["PriceDifference"] = df_final["EndPrice"] - df_final["StartPrice"]

df_final_filtered = df_final[["Symbol", "StartPrice", "EndPrice", "AnnualReturn", "PriceDifference", "totalEsg"]]

df_final_filtered["totalEsg"] = pd.to_numeric(df_final_filtered["totalEsg"], errors="coerce")
df_final_filtered["AnnualReturn"] = pd.to_numeric(df_final_filtered["AnnualReturn"], errors="coerce")
df_final_filtered["PriceDifference"] = pd.to_numeric(df_final_filtered["PriceDifference"], errors="coerce")


print(df_final_filtered.head())


## Summary Statistics for Annual Returns

- A custom function named `describe_return()` was used to calculate detailed summary statistics for the `AnnualReturn` variable.

- This function takes a DataFrame and a label as inputs, and returns the following metrics for the selected column:
  - **Count**: Total number of non-null observations.
  - **Minimum and Maximum**: The lowest and highest return values.
  - **Range**: Difference between maximum and minimum returns.
  - **Mean**: The average return across all companies.
  - **Median**: The middle value in the distribution.
  - **Variance and Standard Deviation**: Measures of return volatility.
  - **Skewness**: Indicates the asymmetry of the return distribution.
  - **Kurtosis**: Measures the "peakedness" or presence of outliers in the data.

- These statistics provide insight into the overall shape, central tendency, and variability of stock performance across the S&P 500 companies.

In [ ]:
## Summary Statistics for Annual Returns

annual_stats = describe_return(df_final_filtered, "Annual Return Statics")
print(annual_stats)

## Descriptive Statistics for Annual Return (All Companies)

The following table summarizes the statistical characteristics of annual returns across all S&P 500 companies in the dataset:

| Metric       | Value     | Interpretation |
|--------------|-----------|----------------|
| **Count**    | 426       | Total number of companies included in the analysis. |
| **Min**      | -0.75     | The worst-performing stock lost 75.1% over the period. |
| **Max**      | 7.34      | The best-performing stock gained 734%, indicating the presence of extreme outliers. |
| **Range**    | 8.09      | The difference between the minimum and maximum returns, showing a wide performance spread. |
| **Mean**     | 0.31      | Average annual return was 30.7%, which may reflect the strong market performance for this period. |
| **Median**   | 0.21      | The median return was 20.7%, suggesting a right-skewed distribution. |
| **Variance** | 0.3168    | Shows the average squared deviation from the mean return. |
| **Std Dev**  | 0.5628    | A standard deviation of 56% indicates high volatility in returns. |
| **Skewness** | 5.39      | Indicates strong positive skewness—most companies had moderate returns, with a few outliers on the high end. |
| **Kurtosis** | 58.02     | Significantly higher than normal (which is 3), pointing to a very peaked distribution with many outliers. |

---

### Interpretation

> The distribution of annual returns is highly **right-skewed**. This means that while most companies earned modest returns, a small number achieved extremely high gains. 
> The standard deviation and variance indicate a wide spread in performance, and the high kurtosis value (58.02) confirms the presence of extreme outliers. 

- As a result, the dataset deviates significantly from a normal distribution, which should be taken into account when performing statistical tests and interpreting comparisons between ESG score groups.


### Histogram of Annual Return – Interpretation

The histogram of annual returns visually reinforces the statistical summary presented earlier:

- The **distribution is clearly right-skewed**, which aligns with the calculated **skewness value of 5.39**. This means most companies earned moderate returns, while a few achieved extremely high gains.
- The **majority of returns fall between 0% and 1%**, which is consistent with the **median value of 20.7%**, indicating that a typical firm performed reasonably well during the period.
- The **presence of extreme outliers**, with returns reaching as high as **+734%**, visually confirms the very high **kurtosis value of 58.02**. These extreme cases have a significant influence on the mean and the spread of the data.
- The **range** of returns (approximately 809%) and the **standard deviation (56%)** reflect the broad variation in performance among companies, which is clearly observable in the wide spread of the histogram.

> Overall, the histogram provides strong visual support for the conclusion that the distribution of annual returns is highly non-normal, with significant skewness and the presence of extreme values. This should be taken into account when choosing appropriate statistical tests.


In [ ]:
# Histogram of Annual Return, to see how distrubuted the data is

plt.figure(figsize=(8, 6))
plt.hist(df_final_filtered["AnnualReturn"], bins=30, edgecolor='black', alpha=0.7)
plt.title("Histogram of Annual Return")
plt.xlabel("Annual Return")
plt.ylabel("Frekans")
plt.grid(True)
plt.xlim(-1, 8)
plt.show()

### Scatter Plot: Total ESG Score vs. Annual Return – Interpretation

This scatter plot visualizes the relationship between companies’ **total ESG scores** and their corresponding **annual returns**.
- The plot shows that the data points are **widely scattered**, with **no clear linear trend** between ESG scores and stock performance.
- The lack of a strong upward or downward pattern suggests that companies with higher ESG scores do **not necessarily achieve higher (or lower) returns** within the observed period.
> Therefore, the scatter plot reinforces the hypothesis test result that **ESG scores alone do not have a measurable linear impact on stock performance** across the S&P 500 companies for the selected time frame.


In [ ]:
### Scatter Plot: Total ESG Score vs. Annual Return

plt.figure(figsize=(8, 6))
plt.scatter(df_final_filtered["totalEsg"], df_final_filtered["AnnualReturn"], alpha=0.7, edgecolors='k')
plt.title("Total ESG Score vs Annual Return")
plt.xlabel("Total ESG Score")
plt.ylabel("Annual Return")
plt.grid(True)
plt.show()

###  Scatter Plot with Regression Line: Total ESG Score vs. Annual Return

This scatter plot extends the previous visualization by including a **regression line**, which helps to evaluate the potential linear relationship between **Total ESG Score** and **Annual Return**.

- The regression line is a **line of best fit**, generated using **ordinary least squares (OLS)** regression. It estimates the average trend of how annual return changes in relation to ESG scores.
- Visually, the regression line is **almost flat**, suggesting that changes in ESG score have little to no linear effect on annual return.
- Adding a regression line is helpful when:
  - We want to confirm or visually validate statistical results (e.g., correlation).
  - We aim to detect any general upward or downward trend that may not be obvious from the raw scatter plot alone.
  - We need to support the hypothesis testing with a simple predictive model.

> In conclusion, the regression line provides additional evidence that there is no meaningful linear association between ESG scores and stock returns in this dataset.


In [ ]:
###  Scatter Plot with Regression Line: Total ESG Score vs. Annual Return

plt.figure(figsize=(8, 6))
sns.regplot(x="totalEsg", y="AnnualReturn", data=df_final_filtered, scatter_kws={"alpha":0.6})
plt.title("Total ESG Score vs Annual Return (Regression Line)")
plt.xlabel("Total ESG Score")
plt.ylabel("Annual Return")
plt.grid(True)
plt.show()

### Hypothesis Testing: Pearson Correlation Test: ESG Score vs Annual Return

This function calculates the **Pearson correlation coefficient** (r) and the corresponding **p-value**, determining whether there is a statistically significant linear relationship between the total ESG score and annual return.

#### Test Results:
- **Pearson r value**: -0.0714  
- **p-value**: 0.1411

#### Interpretation:
- The correlation coefficient is very close to **zero** and **negative**, indicating a **very weak and negative relationship** between ESG scores and stock returns.
- The **p-value is greater than 0.05**, meaning we **fail to reject the null hypothesis (H₀)**.
- Therefore, there is **no statistically significant evidence** of a linear relationship between a company's ESG performance and its stock's annual return in the observed period.

---

### How This Aligns with the Overall Analysis

- The **scatter plot** showed no visible pattern or clustering trend.
- The **regression line** in the extended scatter plot was nearly flat, reinforcing the absence of a linear relationship.
- The **descriptive statistics** revealed a **highly skewed and peaked distribution** (Skewness = 5.39, Kurtosis = 58.02), suggesting that the mean was significantly influenced by a few extreme performers.
- The **histogram** further confirmed that most companies had moderate returns, while a small number exhibited outlier performance.

In [ ]:
# Calculate Pearson Correlation and p-value:

r, p_value = stats.pearsonr(df_final_filtered['totalEsg'], df_final_filtered['AnnualReturn'])

print("Pearson Correlation Test")
print(f"Pearson r value       : {r:.4f}")
print(f"p-value                : {p_value:.4f}")

if p_value <= 0.05:
    print("Reject H0: There is a relationship between ESG scores and stock prices (annual return).")
else:
    print(" Fail to Reject H0: There is no relationship between ESG scores and stock prices (annual return).")


### Final Conclusion

Based on the combination of visual analysis, descriptive statistics, and hypothesis testing, we conclude that **there is no meaningful linear correlation** between Total ESG Scores and Annual Stock Returns among S&P 500 companies in this dataset.  
While ESG factors remain crucial from an ethical and regulatory standpoint, this analysis suggests that **their direct, short-term financial impact on stock performance is not supported**.